In [1]:
import os
import ssl
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

from config import Config, Device
from datasets import BalancedMRIDataset
from models import InceptionV3
from trainer import Trainer
from tester import Tester

/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = Device.device
print(device)

mps


In [3]:
data_path = os.path.join(os.getcwd(), "data")
labels_path = "train.csv"

batch_size = Config.batch_size
num_epochs = Config.num_epochs
learning_rate = Config.learning_rate
mean = Config.mean  # mean of the entire datasaet
std = Config.std  # std of the entire dataaset
image_size = 224

In [4]:
resclaed_mean = round(mean/255, 4)  # re-scale the actual mean
rescaled_std = round(std/255, 4)  # re-scale the actual std

train_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=10),
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[resclaed_mean], std=[rescaled_std])
])

augment_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    # transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[resclaed_mean], std=[rescaled_std])
])

test_transforms = transforms.Compose([
    # transforms.Lambda(lambda img: img.astype(np.float32)),
    transforms.ToTensor(),
    transforms.Resize((299, 299)),
    transforms.Normalize(mean=[resclaed_mean], std=[rescaled_std])
])

In [5]:

train_dataset = BalancedMRIDataset(
    data_path,
    labels_path,
    split='train',
    transform=train_transforms,
    augment_transform=augment_transforms,
    augment=True,
    max_slices=20
)

val_dataset = BalancedMRIDataset(
    data_path,
    labels_path,
    split='val',
    transform=test_transforms,
    max_slices=20
)

test_dataset = BalancedMRIDataset(
    data_path,
    labels_path,
    split='test',
    transform=test_transforms,
    max_slices=20
)

train_dl = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=32)
test_dl = DataLoader(test_dataset, batch_size=32)

In [6]:
data_, label_ = next(iter(train_dl))
data_.size()

torch.Size([32, 20, 299, 299])

In [7]:
ssl._create_default_https_context = ssl._create_stdlib_context

In [8]:
model = InceptionV3().to(device=device)

In [13]:
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [14]:
model_name = model.__class__.__name__
model_name

'Inception3'

In [17]:
trainer = Trainer(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_dl=train_dl,
    val_dl=val_dl,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    device=device,
    num_epochs=100,
    patience=100,
    threshold=0.5,
    save_path=f"saved_models/{model_name}.pth"
)

# Start training
trainer.train()

Epoch 1/100


100%|██████████| 89/89 [14:38<00:00,  9.87s/it]


Confusion Matrix:
[[487  61]
 [ 58  20]]
Train Loss: 767.3479, Train Accuracy: 0.8245
Val Loss: 436.0755, Val Accuracy: 0.8099
Precision: 0.2469, Recall: 0.2564, AUC: 0.5725, Avg Metric: 0.4377
Epoch 2/100


100%|██████████| 89/89 [14:50<00:00, 10.00s/it]


Confusion Matrix:
[[ 92 456]
 [ 15  63]]
Train Loss: 689.7377, Train Accuracy: 0.9177
Val Loss: 538.7695, Val Accuracy: 0.2476
Precision: 0.1214, Recall: 0.8077, AUC: 0.4878, Avg Metric: 0.3922
Epoch 3/100


...

Confusion Matrix:
[[547   1]
 [ 61  17]]
Train Loss: 25.5949, Train Accuracy: 1.0000
Val Loss: 330.8820, Val Accuracy: 0.9010
Precision: 0.9444, Recall: 0.2179, AUC: 0.6081, Avg Metric: 0.6878
Epoch 45/100


100%|██████████| 89/89 [14:25<00:00,  9.73s/it]


Confusion Matrix:
[[361 187]
 [ 29  49]]
Train Loss: 28.7162, Train Accuracy: 1.0000
Val Loss: 980.5257, Val Accuracy: 0.6550
Precision: 0.2076, Recall: 0.6282, AUC: 0.6435, Avg Metric: 0.4969
Epoch 46/100


  0%|          | 0/89 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(trainer.train_losses, "bo", label="Training loss")
plt.plot(trainer.val_losses, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()

In [15]:
model.load_state_dict(torch.load(f"saved_models/{model_name}.pth"))

/var/folders/gf/rw5sv1qs443ggnx74bgbk3tc0000gn/T/ipykernel_4395/324707306.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"saved_models

<All keys matched successfully>

In [16]:
tester = Tester(
    model=model,
    criterion=criterion,
    test_dl=test_dl,
    test_dataset=test_dataset,
    device=device,
    threshold=0.5
)

tester.test()

Test Accuracy: 0.8802, Precision: 0.6154, Recall: 0.1026, AUC: 0.5467, Avg Metric: 0.5327
